In [1]:
import matplotlib.pyplot as plt

from Ballot import Ballot
from DefaultConfigOptions import *
from PartyPrimaryElection import PartyPrimaryElection
from ElectionResult import ElectionResult
from DistrictData import DistrictVotingRecord, DistrictData
from InstantRunoffElection import InstantRunoffElection
from HeadToHeadElection import HeadToHeadElection
from Population import Population
from NDPopulation import NDPopulation
from typing import List, Set, Callable
from Election import Election
from CandidateModel import CandidateModel
import tensorflow as tf
import tensorboard as tb

/Users/robbie/work/rcv-tensorflow/ActionMemory.py:49: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (self.depth == state.shape[1], "depth must match")


In [2]:


class ElectionConstructor:
    def __init__(self, constructor: Callable[[List[Ballot], Set[Candidate]], Election], name: str):
        self.constructor = constructor
        self.name = name

    def run(self, ballots: List[Ballot], candidates: Set[Candidate]) -> ElectionResult:
        e = self.constructor(ballots, candidates)
        return e.result()


def construct_irv(ballots: List[Ballot], candidates: Set[Candidate]):
    return InstantRunoffElection(ballots, candidates)


def construct_h2h(ballots: List[Ballot], candidates: Set[Candidate]):
    return HeadToHeadElection(ballots, candidates)

In [3]:
class Sample:
    def __init__(self, opponents: List[Candidate], candidate: Candidate):
        self.opponents = opponents.copy()
        self.candidate = candidate

In [4]:
candidate_model = CandidateModel(ideology_bins=64, ideology_dim=1, n_hidden=256, batch_size=128, learn_rate= 0.0001)

class Stats:
    def __init__(self):
        self.model_count = 1e-5
        self.model_winners = 0
        self.random_count = 1e-5
        self.random_winners = 0

    def add_random(self):
        self.random_count += 1

    def add_model(self):
        self.model_count += 1

    def add_random_winner(self):
        self.random_winners += 1

    def add_model_winner(self):
        self.model_winners += 1

    def print(self, step: int):
        print("%7d random %6d/%6d %.2f" % (step, self.random_count, self.random_winners, 100 * self.random_winners / self.random_count), end='')
        print(" model %6d/%6d %.2f" % (self.model_count, self.model_winners, 100 * self.model_winners / self.model_count))

stats = Stats()

In [5]:
def run_election(process:  ElectionConstructor):
    pop = NDPopulation(np.array([0]), np.array([40]), Independents)
    voters = pop.generate_voters(1000)

    samples: List[Sample] = []
    candidates: List[Candidate] = []
    is_random = {}
    for i in range(5):
        if np.random.randint(0, 2) == 0 and candidate_model.ready():
            ideology = Ideology(candidate_model.choose_ideology(candidates))
            stats.add_model()
            c = Candidate("c-" + str(i), Independents, ideology, 0)
            is_random[c]  = False
        else:
            ideology = voters[i].ideology
            c = Candidate("c-" + str(i), Independents, ideology, 0)
            is_random[c]  = True
            if candidate_model.ready():
                stats.add_random()

        samples += [Sample(candidates, c)]
        candidates += [c]

    # candidates.append(Candidate("V", Independents, Ideology(np.random.normal(scale=[1.0, 1.0])), quality=0))

    ballots = [Ballot(v, candidates, default_election_config) for v in voters]
    result = process.run(ballots, set(candidates))
    winner = result.winner()

    if candidate_model.ready():
        if is_random[winner]:
            stats.add_random_winner()
        else:
            stats.add_model_winner()

    for s in samples:
        candidate_model.add_sample_from_candidates(s.candidate, s.opponents, result.winner())


In [ ]:
first = True
for batch in range(1000 * 1000):
    if batch < 1024 or random.randrange(0, 10) == 0:
        run_election(ElectionConstructor(construct_irv, "Instant Runoff"))

    if candidate_model.ready():
        if first:
            print("starting to train")
            first = False

        candidate_model.train(128)
        if candidate_model.global_step % 1000 == 0:
            stats.print(candidate_model.global_step)

starting to train
   1000 random     58/     6 10.34 model     52/    16 30.77
   2000 random     94/     6 6.38 model     91/    31 34.07
   3000 random    137/    11 8.03 model    138/    44 31.88
   4000 random    203/    16 7.88 model    197/    64 32.49
   5000 random    259/    23 8.88 model    246/    78 31.71
   6000 random    326/    30 9.20 model    299/    95 31.77
   7000 random    378/    34 8.99 model    347/   111 31.99
   8000 random    418/    35 8.37 model    387/   126 32.56
   9000 random    470/    43 9.15 model    425/   136 32.00
  10000 random    523/    50 9.56 model    467/   148 31.69
  11000 random    561/    51 9.09 model    514/   164 31.91
  12000 random    618/    55 8.90 model    572/   183 31.99
  13000 random    664/    63 9.49 model    626/   195 31.15
  14000 random    695/    68 9.78 model    645/   200 31.01
  15000 random    734/    73 9.95 model    711/   216 30.38
  16000 random    791/    78 9.86 model    769/   234 30.43
  17000 random    850